In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import tensorflow as tf
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

np.random.seed(1234)
tf.random.set_seed(1234)
layer=[3,32,32,32,32,32,2]

In [9]:
class PINN(Model):
    def __init__(self):
        super(PINN,self).__init__()
        self.lambda_1=tf.Variable([0.0],dtype=tf.float64,name="lambda_1",trainable=True)
        self.lambda_2=tf.Variable([0.0],dtype=tf.float64,name="lambda_2",trainable=True)
        
        self.model=Sequential()
        self.model.add(Flatten(input_shape=(3,1)))
        self.model.add(Dense(layer[0],activation='tanh',name="tDense_0"))
    
        for i in range(1,len(layer)-1):
            self.model.add(Dense(layer[i],activation='tanh',name="Dense_{}".format(i)))
        self.model.add(Dense(layer[-1],name='Dense_end'))
        
        self.optimizer=tf.optimizers.Adam(lr=1e-3)
        
    def call(self,X):
        return self.model.call(X)
    
    def predict(self,x,y,t):
        lambda_1=self.lambda_1
        lambda_2=self.lambda_2
        
        x=tf.Variable(x,name="temp_x",dtype=tf.float64)
        y=tf.Variable(y,name="temp_y",dtype=tf.float64)
        t=tf.Variable(t,name="temp_t",dtype=tf.float64)
        with tf.GradientTape(persistent=True) as Tape3:
            with tf.GradientTape(persistent=True) as Tape2:
                with tf.GradientTape(persistent=True) as Tape1:
                    X=tf.concat([x,y,t],1)
                    psi_and_p=self.call(X)
                    psi=psi_and_p[:,0:1]
                    p=psi_and_p[:,1:2]
        
                u=Tape1.gradient(psi,x)
                v=-Tape1.gradient(psi,y)
    
                p_x=Tape1.gradient(p,x)
                p_y=Tape1.gradient(p,y)
            
            u_t=Tape2.gradient(u,t)
            u_x=Tape2.gradient(u,x)
            u_y=Tape2.gradient(u,y)

            v_t=Tape2.gradient(v,t)
            v_x=Tape2.gradient(v,x)
            v_y=Tape2.gradient(v,y)

        u_xx=Tape3.gradient(u_x,x)
        u_yy=Tape3.gradient(u_y,y)

        v_xx=Tape3.gradient(v_x,x)
        v_yy=Tape3.gradient(v_y,y)
    
    
        del Tape1
        del Tape2
        del Tape3
        
      #  print(u_t.dtype,lambda_1.dtype,u_y.dtype,lambda_2.dtype,u_xx.dtype,u_yy.dtype)

        f_u=u_t+lambda_1*(u*u_x+v*u_y)+p_x-lambda_2*(u_xx+u_yy) 
        f_v=v_t+lambda_1*(u*v_x+v*v_y)+p_y-lambda_2*(v_xx+v_yy)
        
        p=tf.cast(p,dtype=tf.float64)
        return u,v,p,f_u,f_v
    
    def BC(self,u,v,u_real,v_real):
        
        return tf.reduce_mean(tf.square(u-u_real)+tf.square(v-v_real)) 
    
    def PDE(self,f_u,f_v):
        return tf.reduce_mean(tf.square(f_u)+tf.square(f_v))
    
    def loss_function(self,u,v,u_real,v_real,f_u,f_v):
        return self.BC(u,v,u_real,v_real)+self.PDE(f_u,f_v)
    
    def run_optimizer(self,x,y,t,u_real,v_real):
        optimizer=self.optimizer
        with tf.GradientTape() as Tape:
            u,v,p,f_u,f_v=self.predict(x,y,t)
            loss=self.loss_function(u,v,u_real,v_real,f_u,f_v)
        
        trainable_variables=self.trainable_variables
        gradients=Tape.gradient(loss,trainable_variables)
        optimizer.apply_gradients(zip(gradients,trainable_variables))
        
    def error(self,u,v,p,u_real,v_real,p_real):       
#        print(u.dtype,v.dtype,p.dtype,u_real.dtype,v_real.dtype,p_real.dtype)
        
        e_u=tf.reduce_mean(tf.square(u-u_real))
        e_v=tf.reduce_mean(tf.square(v-v_real))
        e_p=tf.reduce_mean(tf.square(p-p_real))
        return e_u,e_v,e_p
    
pinn=PINN()

In [10]:
N_train=5000
training_steps=500
batch_size=32
display_step=10

In [11]:
data=scipy.io.loadmat('../input/naiverstokes/cylinder_nektar_wake.mat')
U_star=data['U_star']
P_star=data['p_star']
t_star=data['t']
X_star=data['X_star']
    
N=X_star.shape[0]
T=t_star.shape[0]
    
XX=np.tile(X_star[:,0:1], (1,T))
YY=np.tile(X_star[:,1:2], (1,T))
TT=np.tile(t_star, (1,N)).T
    
UU=U_star[:,0,:]
VV=U_star[:,1,:]
PP=P_star
    
x=XX.flatten()[:,None]
y=YY.flatten()[:,None]
t=TT.flatten()[:,None]
    
u=UU.flatten()[:,None]
v=VV.flatten()[:,None]
p=PP.flatten()[:,None]

In [12]:
idx=np.random.choice(N*T,N_train,replace=False)
x_train=x[idx,:]
y_train=y[idx,:]
t_train=t[idx,:]
u_train=u[idx,:]
v_train=v[idx,:]
p_train=p[idx,:]

snap=np.array([100])
x_star=X_star[:,0:1]
y_star=X_star[:,1:2]
t_star=TT[:,snap]
    
u_star=U_star[:,0,snap]
v_star=U_star[:,1,snap]
p_star=P_star[:,snap]

losschange=[]
e_uchange=[]
e_vchange=[]
e_pchange=[]

In [13]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train,t_train,u_train,v_train,p_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [14]:
for step,(batch_x,batch_y,batch_t,batch_u,batch_v,batch_p) in enumerate(train_data.take(training_steps),1):
    pinn.run_optimizer(batch_x,batch_y,batch_t,batch_u,batch_v)
    
    if step%display_step==0:
        u,v,p,f_u,f_v=pinn.predict(batch_x,batch_y,batch_t)
        loss=pinn.loss_function(u,v,batch_u,batch_v,f_u,f_v)
        e_u,e_v,e_p=pinn.error(u,v,p,batch_u,batch_v,batch_p)
        
        losschange.append(loss)
        e_uchange.append(e_u)
        e_vchange.append(e_v)
        e_pchange.append(e_p)
        
        print("step: %i, loss: %f，error:u:%f v:%f p:%f" %(step,loss,e_u,e_v,e_p))

In [15]:
tf.print(pinn.lambda_1,pinn.lambda_2)

In [16]:
u_pred,v_pred,p_pred,f_u,f_v=pinn.predict(x_star,y_star,t_star)
e_u,e_v,e_p=pinn.error(u_pred,v_pred,p_pred,u_star,v_star,p_star)

In [17]:
e_u,e_v,e_p

In [18]:
X=np.linspace(0,training_steps,np.array(losschange).shape[0])

In [19]:
np.array(losschange).shape[0]

In [20]:
np.array(losschange).shape[0]

In [23]:
plt.figure(dpi=240,figsize=(6,4))
plt.plot(X,losschange,color='skyblue',ls='-.',label='loss')
plt.plot(X,e_uchange,color='green', label='u')
plt.plot(X,e_vchange,color='red', label='v')
plt.plot(X,e_pchange,color='black', label='p')
plt.legend()
 
plt.xlabel('step')
plt.ylabel('error/loss')
plt.show()
plt.savefig('PINN_NS.jpg')